In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

Pytorch package

1. torch
메인 네임스페이스로 텐서 등 다양한 수학 함수가 포함되어져 있으며 Numpy와 유사한 구조를 가진다.


In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



https://pytorch.org/vision/stable/datasets.html

root : 디렉토리가 존제하거나 다운로드가 true일 때 저장 될 데이터 set의 루트 디렉토리

train : True면 training set에 데이터를 생성하고 False면 test set에 데이터를 생성한다.

dawnload : True이면 인터넷에서 데이터를 다운로드 받아와서 root 에 넣는다. 이미 다운로드 된 상태면 다운로드 받지 않는다.

transform : 입력 샘플을 전환하고 반환한다. 



In [ ]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print("Shape of X [N,C,H,W]:", X.shape)
  print("shape of y", y.shape, y.dtype)
  break

Shape of X [N,C,H,W]: torch.Size([64, 1, 28, 28])
shape of y torch.Size([64]) torch.int64


N C H W : n-count, height, width, channel의 약자

.shape : 텐서의 크기 확인 가능

.dtype : 자료형 출력


*모델만들기*
PyTorch에서 신경망 모델은 nn.Module을 상속받는 클래스(class)를 생성하여 정의

In [ ]:
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usin {device} device")

# 모델정의
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)
  

Usin cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


문서를 그대로 따라치면

```
name 'nn' is not defined
```
오류가 발생한다


```
import torch.nn as nn
```
를 상단에 입력해준다.




모델 매개변수 최적화하기

모델을 학습하려면 손실 함수(loss function)와 옵티마이저(optimizer)가 필요하다.

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X,y) in enumerate(dataloader):
    # 예측오류 계산
    pred = model(X)
    loss = loss_fn(pred, y)

    # 역전파
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss : {loss:>7f} [{current:>5d}/{size:>5d}]")


loss_fn : 손실함수

optimizer : 옵티마이저

In [ ]:
def test(dataloader, model,loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X,y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn (pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \m Accuracy: {100*correct:>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n--------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1
--------------
loss : 2.156430 [    0/60000]
loss : 2.148228 [ 6400/60000]
loss : 2.089865 [12800/60000]
loss : 2.108568 [19200/60000]
loss : 2.046523 [25600/60000]
loss : 1.987397 [32000/60000]
loss : 2.023722 [38400/60000]
loss : 1.943538 [44800/60000]
loss : 1.943945 [51200/60000]
loss : 1.873286 [57600/60000]
Test Error: \m Accuracy: 55.0%, Avg loss: 1.868064 

Epoch 2
--------------
loss : 1.903283 [    0/60000]
loss : 1.875483 [ 6400/60000]
loss : 1.758795 [12800/60000]
loss : 1.801598 [19200/60000]
loss : 1.671623 [25600/60000]
loss : 1.633805 [32000/60000]
loss : 1.662641 [38400/60000]
loss : 1.566276 [44800/60000]
loss : 1.586169 [51200/60000]
loss : 1.484316 [57600/60000]
Test Error: \m Accuracy: 61.1%, Avg loss: 1.498525 

Epoch 3
--------------
loss : 1.569046 [    0/60000]
loss : 1.537781 [ 6400/60000]
loss : 1.389957 [12800/60000]
loss : 1.461343 [19200/60000]
loss : 1.322886 [25600/60000]
loss : 1.331922 [32000/60000]
loss : 1.350538 [38400/60000]
loss : 1.278742

epochs : 전체 데이터 set에 대한 한번의 학습 과정...이라고 생각하면 될 것 같다. 반복단계

모델 저장하기
- 모델을 저장하는 일반적인 방법은 내부 상태 사전을 직렬화(serialize) 하는 것이다

In [ ]:
torch.save(model.state_dict(),"model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


모델 불러오기
- 모델 구조를 다시 만들고 상태 사전을 모델에 불러오는 과정 포함

In [ ]:
model = NeuralNetwork()
# 내가 바로 위에서 저장한거
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
